In [1]:
search_for = 139
start_from = 0
threads = 10
thread = 7
log_freq = 10000



#in this cell, for a given lattice, we compute all forbidden nodes, 
#i.e. those nodes whose Voronoi cell is at a distance less than doubled covering radius

#An* represented in m=n+1 as sum x_i=0 with x_i\equiv to each other mod m. 
#Forbidden nodes will be recorded w.r.t the basis (-n,1,1,..) and permutations

n=5 #lattice dim
m=n+1 #space dim




R = sqrt(n*(n+1)*(n+2)/12) #covering radius

#first we generate all nodes with norm <=4R, and with norm<=2R
t=1 #.3 #can increase a bit for random from non-forbidden
short = []
vshort = []
K = ceil(t*4*R)
K2 = (t*4*R)^2
Kv = (2*R)^2

indexes = [] #arrays of integers between -K and K having the same residue modulo m 
for rem in range(m):
    res = []
    for i in range(-K,K+1):
        if i%m==rem:
            res.append(i)
    indexes.append(res)

for c in range(0,K+1):
    for coefs in mrange_iter([indexes[c%m] for i in range(m-2)]):
        p = [c]+list(coefs)+[-c-sum(coefs)]
        is_sorted = True
        for i in range(m-1):
            if p[i]<p[i+1]:
                is_sorted = False
                break
        if is_sorted:
            norm2 = sum((p[i])^2 for i in range(m))
            if norm2>0:
                if norm2 <= Kv:
                    vshort.append(p)
                if norm2 <= K2:
                    short.append(p)           

print("very short:",len(vshort))
print("short:",len(short))


from time import time
tm = time()


#defining the cone arising due to symmetry - it suffices to construct Voronoi cell in it
ineqs = [] 
for i in range(m-1):
    toadd = [0]*(m+1)
    toadd[i+1]=1
    toadd[i+2]=-1
    ineqs.append(toadd) 
    
ineqs += [[0]+[1]*m,[0]+[-1]*m] #we are in the hyperplane where all coords sum to zero

#now Voronoi cell planes
for p in vshort: #suffices to look at nodes at most 2R
    ineqs.append([sum((p[i])^2 for i in range(m))]+[-2*p[i] for i in range(m)])

V = Polyhedron(ieqs=ineqs)
    
print(time()-tm)    

### orthogonal projection affine operator onto (improper) non-empty polytope 
zerom = matrix(m)
def projp(P):
    nv = P.n_vertices()
    z = vector(P.vertices()[0])
    if nv==1:
        return zerom,z
    A = matrix([vector(P.vertices()[i])-z for i in range(1,nv)]).transpose()
    return A*A.pseudoinverse(),z


forb = []

for d in range(0,n): #dim of face
    for ff in V.faces(d):
        proj = projp(ff)
        for p in short:
            if p not in forb:
                x = vector([t/2 for t in p]) #dist from 1/2 of a node to the Voronoi cell equals 1/2 of the dist between cells at zero and at the node
                proj_pnt = proj[0]*(x-proj[1])+proj[1]
                if proj_pnt in ff.as_polyhedron():
                    dist = (x-proj_pnt).norm()
                    if dist<R:
                        forb.append(p)

print("forbidden:",len(forb))

#expand by the basis (-n,1,1,..) and permutations
def basis_coefs(x):
    y = x[0:(m-1)]
    sy = sum(y)
    return [(-sy-y[i])/m for i in range(m-1)]

### forbd = forb with possible permutations
forbd = {}
forbdv = []
for x in forb:
    for t in Permutations(list(x)):
        c = vector(basis_coefs(t))
        if str(c) not in forbd:
            forbd[str(c)]=0
            if sum(c)>=0:
                forbdv.append(c)
print("forbidden without symmetries:",len(forbd))
###





fn = 0
for x in forb:
    norm2 = sum(t^2 for t in x)
    if norm2>fn:
        fn = norm2
print("square L2 norm of forbidden:",fn)


small = 2 #max abs value of a small coefficient
sc = []
for c in mrange_iter([range(-small,small+1) for i in range(n)]):
    if sum(c)>=0:
        sc.append(vector(c))
sc.sort(key=lambda i: sum(x*x for x in i)) #abs(x) in place of x*x seems a bit slower
sc = sc[1:] #removing zero

print("small:",len(sc))

order = [(-1)^i*(1/4+i/2)-1/4 for i in range(1000)] #[0,-1,1,-2,2,...]

def check_sublattice(basis,target=Infinity): 
    A = matrix(basis) #rows
    #A = A.LLL() #benchmark to see if helps
    A = A.transpose() #columns

#     Ad = abs(A.det())
#     if Ad==0 or Ad>=target:
#         return Infinity
        
    for coefs in sc:
        if str(A*coefs) in forbd:
            return Infinity

    Ai = A.inverse()
    Ai_norms = [sum(t^2 for t in Ai[i]) for i in range(n)]
    M = []
    Mp = 1
    Mps = 1
    for i in range(n):
        mi = floor(sqrt(fn*Ai_norms[i]))
        M.append(mi)
        Mp *= (2*mi+1)
        Mps *= (2*min(small,mi)+1)
        
    if max(M)>small: #otherwise already considered
        if Mp-Mps<len(forbd): #faster using A, otherwise A_inv
            for coefs in mrange_iter([order[0:(2*M[i]+1)] for i in range(n)]):
                if sum(coefs)>=0 and max(abs(x) for x in coefs)>small:
                    if str(A*vector(coefs)) in forbd:
                        return Infinity
        else: #check if A_inv times forbidden is not in Z^n
            for x in forbdv:
                y = Ai*x
                allint = True
                for t in y:
                    if t!=floor(t):
                        allint = False
                        break
                if allint:
                    return Infinity   
    
    return 0#Ad



#random/exhaustive sublattice search, skip to the cell starting with good for search using non-forbidden generating vectors of small norm
#each type of a matrix is the numbers on the diagonal
maxd = search_for #bound for diagonal entries
mindet = search_for #minimum determinant
maxdet = search_for #maximum determinant

def all_types(m,d1,d2,n):
    res = []
    if n==1:
        for k in range(d1,min(d2,m)+1):
            res.append([k])
        return res
    for k in range(1,min(d2,m)+1):
        prev = all_types(m,ceil(d1/k),floor(d2/k),n-1)
        for x in prev:
            res.append([k]+x)
    return res

types = all_types(maxd,mindet,maxdet,n)

print("types:",len(types))

def ind_max(t): #given type, lists maxima of indexes to define entries above the diagonal
    return [t[i]^i for i in range(1,n)]


allc = 0
for t in types:
    allc += prod(ind_max(t))
print("all cases:",allc)

from random import randint
def rand_ind(t): #given type, return random indexes
    maxs = ind_max(t)
    return [randint(0,maxs[i]-1) for i in range(n-1)]

def sublat_mat(t,ind): #given type and indexes, return sublattice basis
    res = matrix(n)
    for i in range(n):
        res[i,i] = t[i]
    for i in range(1,n):
        tt = -floor(t[i]/2)
        indc = ind[i-1]
        for j in range(i):
            res[j,i] = tt + (indc % t[i])
            indc = floor(indc / t[i])
    return res

def exh_search():
    tm = time()
    bestv = Infinity
    best = []
    allcnt = 0
    for tt in types:
        print("type:",tt,prod(tt))
        cnt = 0
        maxinds = ind_max(tt)
        cM = prod(maxinds)
        print("cases of this type:",cM)
        for i in range(cM):
            allcnt += 1
            if allcnt%(log_freq*threads)==0:
                print(allcnt,(time()-tm)/3600*(allc-allcnt)/(log_freq*threads))
                tm=time()
            if allcnt>=start_from and allcnt%threads==thread:
                inds = []
                l = i
                for j in range(n-1):
                    inds.append(l%maxinds[j])
                    l=floor(l/maxinds[j])
                mat = sublat_mat(tt,inds)
                cs = check_sublattice(mat)
                if cs==0:
                    cs = prod(tt)
                    if cs<bestv:
                        print("sublattice basis:")
                        print(mat)
                        bestv = cs
                        best = [mat]
                    elif cs==bestv:
                        best.append(mat)
        print("best so far:",len(best))
    print("all cases:",allcnt)
    print("best:",len(best))
    return best





best = exh_search()




very short: 5
short: 39
0.08977437019348145
forbidden: 31
forbidden without symmetries: 1984
square L2 norm of forbidden: 264
small: 1752
types: 5
all cases: 376006121
type: [1, 1, 1, 1, 139] 139
cases of this type: 373301041
100000 12.500909012053393
200000 22.422226225520205
300000 22.802121155525487
400000 20.829384653928145
500000 20.808633343167422
600000 20.539425361607407
700000 21.67160370911457
800000 19.826587847072716
900000 20.15538112950074
1000000 23.665112836725033
1100000 21.410156729454506
1200000 21.453200897712364
1300000 20.222829475587826
1400000 3.966234175484816
1500000 20.94979212940005
1600000 24.97319186025798
1700000 25.11135136813215
1800000 19.612900619062096
1900000 21.22250440454718
2000000 24.860674504712737
2100000 25.42802245140987
2200000 33.53413987065139
2300000 33.57804609942255
2400000 36.89885213273806
2500000 34.18434391837493
2600000 36.53694746101111
2700000 17.148650942195218
2800000 24.285783559638308
2900000 41.375201963692575
3000000 46.27

29800000 12.669909910568137
29900000 32.77755253561431
30000000 37.66095997959229
30100000 35.180722961064454
30200000 37.25822679425108
30300000 33.48139071907229
30400000 37.50165066919162
30500000 33.626384771315365
30600000 36.48442451193412
30700000 32.48630785934035
30800000 33.669750654600115
30900000 16.564922135954074
31000000 25.95806795892193
31100000 31.044927062933866
31200000 35.61114637348732
31300000 32.41998328742691
31400000 35.42250092472642
31500000 32.73292532198937
31600000 34.828880050923
31700000 33.61292563282121
31800000 35.67950676115278
31900000 34.85814324813541
32000000 25.185471954081798
32100000 26.136557164232865
32200000 36.031098174254105
32300000 36.36618384931247
32400000 36.24747861136587
32500000 14.276893621770414
32600000 31.34074615885035
32700000 35.85860240116416
32800000 35.71434313711805
32900000 35.767763965125255
33000000 33.5846284970072
33100000 35.162713562273176
33200000 34.05740673926766
33300000 34.389431609763975
33400000 33.042629

59800000 31.505240747835018
59900000 30.1248156424453
60000000 30.28993387350977
60100000 30.120081585078015
60200000 29.223151734986867
60300000 28.99384410839731
60400000 24.355901395841865
60500000 12.471243735948834
60600000 28.747073285776764
60700000 29.092472655950818
60800000 28.90446086371907
60900000 29.172261704754128
61000000 30.16214820676893
61100000 29.6602225908089
61200000 30.092752226882926
61300000 29.898847701562904
61400000 13.37825526966764
61500000 32.666646786331405
61600000 31.094675494544855
61700000 32.19889586004683
61800000 31.399296766691624
61900000 34.443817430441435
62000000 32.31967598836677
62100000 34.14837553317743
62200000 27.227548496329607
62300000 16.01769691713391
62400000 31.730916484083043
62500000 31.77982693985726
62600000 31.52601525546659
62700000 32.32720397684238
62800000 30.41229202484551
62900000 28.78876992473025
63000000 29.201039561411353
63100000 20.98865165021911
63200000 16.536663980271477
63300000 28.402957255449557
63400000 29

89600000 27.106949857850914
89700000 27.11954644232073
89800000 26.856976517769716
89900000 26.546090445942255
90000000 9.229228786737199
90100000 24.780347698655213
90200000 26.317769145846036
90300000 26.866112912795224
90400000 26.075323817531917
90500000 26.80207738711351
90600000 26.237207212933082
90700000 11.146791180130174
90800000 27.59361564806406
90900000 28.78876735781551
91000000 29.34219181965126
91100000 29.176577120865815
91200000 29.677529530180777
91300000 30.402241887154247
91400000 30.85182311977969
91500000 30.07150678837624
91600000 29.98528231640519
91700000 30.020158712879756
91800000 29.266980670299894
91900000 29.342089346058913
92000000 13.969986677667457
92100000 22.2558137999118
92200000 27.387546493994744
92300000 26.99878826362126
92400000 26.46692583625406
92500000 26.88470907482272
92600000 27.514823847321654
92700000 4.70159361006804
92800000 28.74918745634404
92900000 27.076486822422087
93000000 26.091467505635475
93100000 27.50055625168845
93200000 2

118600000 25.325779040717105
118700000 25.530708200356433
118800000 24.844000440729726
118900000 23.829294254850346
119000000 20.74673324008375
119100000 10.982753081263859
119200000 23.57100469092983
119300000 22.72209278013875
119400000 23.649961555629634
119500000 16.827015538139545
119600000 13.673038968237645
119700000 23.50817867867692
119800000 23.446513632036996
119900000 23.38642879502851
120000000 15.180875799379924
120100000 18.300185934512353
120200000 23.688800826326826
120300000 23.403857762178113
120400000 24.968431602543873
120500000 26.280076550381615
120600000 26.351065185585966
120700000 26.29085565992119
120800000 26.164440110897548
120900000 26.17359547005415
121000000 25.719240896568625
121100000 26.21027601604138
121200000 25.78717150904259
121300000 25.00505900147701
121400000 26.06977764145262
121500000 23.95824189322869
121600000 23.598209853462148
121700000 21.35245863375701
121800000 10.066960410231884
121900000 23.335019792502745
122000000 24.46677678361765

...

233200000 14.238029198971281
233300000 14.36961877885564
233400000 15.71796503060084
233500000 15.764517488512837
233600000 14.946963038070168
233700000 15.162979776680325
233800000 13.931230471051984
233900000 15.293984318704712
234000000 15.329345389948282
234100000 14.198475466036992
234200000 14.241932340780629
234300000 13.372333394601084
234400000 13.710276052258411
234500000 13.821054488529086
234600000 13.052835780945113
234700000 4.777117997444225
234800000 14.628028130586982
234900000 12.894210348839245
235000000 7.149815533879161
235100000 9.216933197924178
235200000 14.6627472218761
235300000 11.956942780677423
235400000 5.31574704612005
235500000 12.460674040826872
235600000 14.477284736467594
235700000 12.334112127968467
235800000 15.000427379850878
235900000 13.37313796712619
236000000 14.904417477777097
236100000 15.304904651388437
236200000 15.000469127146161
236300000 14.939223275985778
236400000 16.125590270213365
236500000 13.73610344705942
236600000 16.320332017101

261800000 10.619954019853107
261900000 1.7760957015304115
262000000 10.927447524941192
262100000 10.840059659644984
262200000 10.331452153973531
262300000 10.743570146135701
262400000 6.4449213104309795
262500000 8.296393269866304
262600000 10.792501138427166
262700000 11.101069999775703
262800000 11.897639420993649
262900000 12.313171360423695
263000000 12.041826928049597
263100000 12.168745679611456
263200000 12.64097538217428
263300000 11.693126414150095
263400000 12.828393261177961
263500000 11.248278674722293
263600000 12.210211750746197
263700000 11.31410363781018
263800000 12.012789911703177
263900000 10.297060284157501
264000000 5.17987870352461
264100000 9.491465895867217
264200000 11.307844247445782
264300000 10.069714226840452
264400000 11.238616303727603
264500000 9.476947766970701
264600000 3.416272642042248
264700000 10.683698194932665
264800000 10.780955876139002
264900000 10.431608895751758
265000000 10.832825052756837
265100000 6.412167034540263
265200000 7.54293059564

290800000 9.229038889855698
290900000 8.919892854852694
291000000 8.12298892437334
291100000 8.709510220861867
291200000 8.12334974754812
291300000 8.38516578203294
291400000 4.326649817659472
291500000 6.710328793503022
291600000 7.982750858349601
291700000 8.428257479612968
291800000 8.611631772404134
291900000 7.946529564220954
292000000 9.328891959404087
292100000 8.336796523912678
292200000 3.128115203185343
292300000 8.452573115009132
292400000 10.061689629131735
292500000 8.931157409423687
292600000 9.715514692586053
292700000 9.266560675104316
292800000 9.993084517297998
292900000 8.525764209929697
293000000 10.049493529415376
293100000 8.88801314619469
293200000 8.956064613293526
293300000 6.137357560049569
293400000 5.6277035673547715
293500000 8.605973982459705
293600000 8.258049218748893
293700000 8.249673418670703
293800000 8.434569132297407
293900000 7.72464096338968
294000000 8.261136707600487
294100000 3.0834297694764983
294200000 7.23220333766157
294300000 7.8602256393

320100000 5.3991337774554164
320200000 6.0777383253533035
320300000 5.6592739057855725
320400000 5.679385890479096
320500000 5.23167174084618
320600000 5.859476380222268
320700000 4.925668912706894
320800000 5.446597809830619
320900000 4.424191501572289
321000000 1.9460271206844357
321100000 5.063287777924036
321200000 5.439213726990056
321300000 5.013052535603492
321400000 5.997331032635893
321500000 5.359249310032008
321600000 5.821232952504627
321700000 5.583835009718693
321800000 6.046949081471151
321900000 3.532414565281511
322000000 3.9246374457464346
322100000 5.588489817716468
322200000 5.921003927120156
322300000 5.605446100324068
322400000 5.956489860493422
322500000 5.729018782428231
322600000 5.06327257302052
322700000 2.8473865104841947
322800000 5.616892236260011
322900000 5.949305562967824
323000000 5.45289318330388
323100000 5.597560847510729
323200000 5.093578831700402
323300000 5.692311035075778
323400000 4.822842608605014
323500000 5.353992950709189
323600000 3.89071

349100000 2.9112117784254132
349200000 2.969313393492661
349300000 1.74515164061517
349400000 2.214923435641686
349500000 2.6939339961829862
349600000 2.9537170401165795
349700000 2.6757778249129314
349800000 3.0253156355308635
349900000 2.496173642796827
350000000 2.946657464403385
350100000 2.6446753868444537
350200000 2.87921496586968
350300000 2.5503906136101326
350400000 2.7310429738750215
350500000 0.9127963595780545
350600000 2.2067745703520396
350700000 2.6355137408043996
350800000 2.696260675866213
350900000 2.6876658948656678
351000000 2.577102224843437
351100000 2.58083596089256
351200000 2.600169269056819
351300000 2.701195274280548
351400000 2.4936337889942015
351500000 2.6882348300085566
351600000 2.668157322397889
351700000 0.5746289793798725
351800000 2.5931115116780696
351900000 2.5158982786667723
352000000 0.8764390406573859
352100000 2.4902008461865126
352200000 2.3501743718127486
352300000 2.4032332700482293
352400000 2.2691478251402737
352500000 2.4369507472718763
